In [1]:
import pandas as pd
import numpy as np
import math
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sksurv.preprocessing import OneHotEncoder
from sksurv.linear_model import CoxPHSurvivalAnalysis
from sksurv.metrics import concordance_index_censored
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import SelectKBest
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

In [2]:
df_sct = pd.read_json('data/GoT-screentimes.json')
df_deaths = pd.read_csv('data/character-predictions.csv')

# Data exploration 
**Data sources** 
* 'deaths' dataset from https://www.kaggle.com/mylesoneill/game-of-thrones
* 'screentime' dataset from https://data.world/aendrew/game-of-thrones-screen-times

Taking a first look at the dataset 

In [4]:
def basic_info(df):
    print("Column list: {}".format(df.columns))
    print("Shape: {}".format(df.shape))
    return(df.head())

In [5]:
basic_info(df_sct)

Column list: Index(['name', 'imdbUrl', 'screentime', 'episodes', 'portrayedBy'], dtype='object')
Shape: (191, 5)


,name,imdbUrl,screentime,episodes,portrayedBy
0,Tyrion Lannister,http://www.imdb.com/character/ch0146096/,293.30,54,"{'name': 'Peter Dinklage', 'imdbUrl': 'http://..."
1,Jon Snow,http://www.imdb.com/character/ch0155777/,268.15,49,"{'name': 'Kit Harington', 'imdbUrl': 'http://w..."
2,Daenerys Targaryen,http://www.imdb.com/character/ch0158597/,221.30,49,"{'name': 'Emilia Clarke', 'imdbUrl': 'http://w..."
3,Cersei Lannister,http://www.imdb.com/character/ch0159526/,201.45,52,"{'name': 'Lena Headey', 'imdbUrl': 'http://www..."
4,Sansa Stark,http://www.imdb.com/character/ch0158137/,199.30,47,"{'name': 'Sophie Turner', 'imdbUrl': 'http://w..."


In [6]:
# Checking to see if we have gender as part of this dataset and what else we have in the portrayedBy dict
df_sct['portrayedBy'][0]

{'name': 'Peter Dinklage', 'imdbUrl': 'http://www.imdb.com/name/nm0227759/'}

In [7]:
basic_info(df_deaths)

Column list: Index(['S.No', 'actual', 'pred', 'alive', 'plod', 'name', 'title', 'male',
       'culture', 'dateOfBirth', 'DateoFdeath', 'mother', 'father', 'heir',
       'house', 'spouse', 'book1', 'book2', 'book3', 'book4', 'book5',
       'isAliveMother', 'isAliveFather', 'isAliveHeir', 'isAliveSpouse',
       'isMarried', 'isNoble', 'age', 'numDeadRelations', 'boolDeadRelations',
       'isPopular', 'popularity', 'isAlive'],
      dtype='object')
Shape: (1946, 33)


,S.No,actual,pred,alive,plod,name,title,male,culture,dateOfBirth,...,isAliveHeir,isAliveSpouse,isMarried,isNoble,age,numDeadRelations,boolDeadRelations,isPopular,popularity,isAlive
0,1,0,0,0.054,0.946,Viserys II Targaryen,NaN,1,NaN,NaN,...,0.0,NaN,0,0,NaN,11,1,1,0.605351,0
1,2,1,0,0.387,0.613,Walder Frey,Lord of the Crossing,1,Rivermen,208.0,...,NaN,1.0,1,1,97.0,1,1,1,0.896321,1
2,3,1,0,0.493,0.507,Addison Hill,Ser,1,NaN,NaN,...,NaN,NaN,0,1,NaN,0,0,0,0.267559,1
3,4,0,0,0.076,0.924,Aemma Arryn,Queen,0,NaN,82.0,...,NaN,0.0,1,1,23.0,0,0,0,0.183946,0
4,5,1,1,0.617,0.383,Sylva Santagar,Greenstone,0,Dornish,276.0,...,NaN,1.0,1,1,29.0,0,0,0,0.043478,1


### Combining datasets
The screentime dataset doesn't have much information by itself. Before we do any further exploration, we try to combine the screentime dataset with the deaths dataset based on the name column, after converting all names to lower case.

In [8]:
df_deaths['name'] = df_deaths['name'].str.lower()
df_sct['name'] = df_sct['name'].str.lower()

df_combined = pd.merge(df_deaths, df_sct, on='name', how='left')

print("Number of valid  screentimes in screentime dataset", df_sct['screentime'].notna().value_counts(), 
      "Number of valid screentimes in combined dataset", df_combined['screentime'].notna().value_counts(), 
      sep = '\n\n')

Number of valid  screentimes in screentime dataset

True    191
Name: screentime, dtype: int64

Number of valid screentimes in combined dataset

False    1820
True      126
Name: screentime, dtype: int64


We are missing 191 - 126 = 65 names which are in the screentime dataset but have not been mapped to any name in the deaths dataset. We try to take a look at these names to see possibly why they are not mapped and try to map a bit more of them before moving on

In [9]:
mapped_names = df_combined[df_combined.screentime.notna()].name
unmapped_names = list(df_sct.query("name not in @mapped_names").name)
print(unmapped_names[:15])

["petyr 'littlefinger' baelish", "eddard 'ned' stark", 'brienne of tarth', 'lord varys', "sandor 'the hound' clegane", 'ramsay bolton', 'tormund giantsbane', 'olenna tyrell', 'grand maester pycelle', 'talisa maegyr', 'robert baratheon', 'yara greyjoy', 'khal drogo', 'maester luwin', 'ros']


In [10]:
title_list = ['lord', 'maester', 'grand maester', 'khal', 'ser', 'septa', 'black']

name_dict = {'ramsay bolton': 'ramsay snow',
             'tormund giantsbane': 'tormund',
             'olenna tyrell': 'olenna redwyne',
             'smalljon umber': 'jon umber (smalljon)',
             'greatjon umber': 'jon umber (greatjon)',
             'selyse baratheon': 'selyse florent'
             }

def clean_name(name):
    if('\'' in name):
        nick = name.split('\'')
        cleaned_name = nick[0].rstrip(' ') + nick[-1]
        return(cleaned_name.strip())
    for title in title_list:
        if name.startswith(title):
            return(name.lstrip(title).strip())
    if(name in name_dict.keys()):
        return(name_dict[name])
    return(name)
    
df_sct['name'] = df_sct['name'].apply(clean_name)
df_combined = pd.merge(df_deaths, df_sct, on='name', how='left')

mapped_names = df_combined[df_combined.screentime.notna()].name
unmapped_names = list(df_sct.query("name not in @mapped_names").name)
print(unmapped_names[:10], "\n\nProportion of names mapped = {0:.2f}".format(1-len(unmapped_names)/len(df_sct.name)))

['brienne of tarth', 'talisa maegyr', 'robert baratheon', 'yara greyjoy', 'jaqen hghar', 'ros', 'aemon', 'thoros of myr', 'locke', 'myranda'] 

Proportion of names mapped = 0.76


Having managed to map more than 3/4 th the names, we move on to exploring thie combined dataset

In [11]:
df_combined.drop(columns=['imdbUrl', 'portrayedBy'], inplace=True)

## Is the data suitable for Survival analysis?
While reading about what methods could be best for a problem of this nature, I learnt about survival analysis, and it fits perfectly. I was thinking that it is somehow not entirely reasonable to consider this a regular binary classification problem because the data for the living characters cannot be considered 'complete' and it seems strange to train on alive, dead as just 1, 0 as many of the alive characters might be very close or very far from their deaths and we have no knowledge of this. I had trouble formalizing this concept but I found a formalization in the language of Survival analysis, and it is right-censoring of the data.

More info on survival analysis [here](https://www.cscu.cornell.edu/news/statnews/stnews78.pdf)

**What do we need in the data to be able to use survival analysis?**

*A timeline*: The process in our case is being alive. So the start date is the birth year, and the event of interest is death. The time passed until the start of the process is then clearly the age. We must check if we have the age for most characters which are alive, and at what age characters died.

*Event times*: We would need to have an age of death for dead characters. We need to check how many of them we have this data for.

*Current time*: We need to know where we are in the timeline, i.e. at what time in the game of thrones world this data si collected and if it is well defined and consistent for different characters.

In [12]:
data = df_combined
num_dead = len(data[data.isAlive == 0].index)
num_alive = len(data[data.isAlive == 1].index)
print("There are {} dead and {} living characters".format(num_dead, num_alive))

dead_proportion = num_dead/(num_dead + num_alive)
print("{0:.2f}% of the data is of dead characters".format(100*dead_proportion))

dead_with_age = len(data[(data.actual == 0) & (~data.age.isna())].index)/num_dead
print("{0:.2f}% of the dead characters have known age of death".format(100*dead_with_age))

alive_with_age = len(data[(data.actual == 1) & (~data.age.isna())].index)/num_alive
print("{0:.2f}% of the living characters have known age".format(100*alive_with_age))

There are 495 dead and 1451 living characters
25.44% of the data is of dead characters
32.73% of the dead characters have known age of death
18.68% of the living characters have known age


In [13]:
print("\u001b[30;1m Stats for popular characters \u001b[0m")
data_pop = data[data.isPopular == 1]
num_dead = len(data_pop[data_pop.isAlive == 0].index)
num_alive = len(data_pop[data_pop.isAlive == 1].index)
print("There are {} dead and {} living characters".format(num_dead, num_alive))

dead_proportion = num_dead/(num_dead + num_alive)
print("{0:.2f}% of the data is of dead characters".format(100*dead_proportion))

dead_with_age = len(data_pop[(data_pop.actual == 0) & (~data_pop.age.isna())].index)/num_dead
print("{0:.2f}% of the dead characters have known age of death".format(100*dead_with_age))

alive_with_age = len(data_pop[(data_pop.actual == 1) & (~data_pop.age.isna())].index)/num_alive
print("{0:.2f}% of the living characters have known age".format(100*alive_with_age))

 Stats for popular characters 
There are 60 dead and 55 living characters
52.17% of the data is of dead characters
46.67% of the dead characters have known age of death
69.09% of the living characters have known age


In [14]:
# Getting the 'current year'
alive_with_age = data[(~data.age.isna()) & (data.isAlive == True) & (~data.dateOfBirth.isna())]
print(set([age+DOB for age, DOB in zip(alive_with_age['age'], alive_with_age['dateOfBirth'])]))

{305.0}


So we see that even thought a large proportion of the dataset doesn't have age, we have it for a majority of the popular characters. Since we would mostly be interested in predicting deaths for popular characters, we go ahead with survival analysis. And also because I have not implemented survival analysis before and it would be an interesting exercise to do so.

Since the 'current year' is well defined, we have a proper reference to continue

# Data cleaning
* Since we are doing survival analysis, only look at rows which have a non null age
* Look at columns with multiple similar strings for what should be the same value. This turns out to be a problem only in the 'culture' column
* Remove columns we do not need to train
* OneHotEncode the data
* Fill nulls
* Get the data to be trained on in the foramt required to use scikit-survival, described [here](https://nbviewer.jupyter.org/github/sebp/scikit-survival/blob/master/examples/00-introduction.ipynb)

In [15]:
# From  Shail Daliwala's kernel on Kaggle
cult = {
    'Summer Islands': ['summer islands', 'summer islander', 'summer isles'],
    'Ghiscari': ['ghiscari', 'ghiscaricari',  'ghis'],
    'Asshai': ["asshai'i", 'asshai'],
    'Lysene': ['lysene', 'lyseni'],
    'Andal': ['andal', 'andals'],
    'Braavosi': ['braavosi', 'braavos'],
    'Dornish': ['dornishmen', 'dorne', 'dornish'],
    'Myrish': ['myr', 'myrish', 'myrmen'],
    'Westermen': ['westermen', 'westerman', 'westerlands'],
    'Westerosi': ['westeros', 'westerosi'],
    'Stormlander': ['stormlands', 'stormlander'],
    'Norvoshi': ['norvos', 'norvoshi'],
    'Northmen': ['the north', 'northmen'],
    'Free Folk': ['wildling', 'first men', 'free folk'],
    'Qartheen': ['qartheen', 'qarth'],
    'Reach': ['the reach', 'reach', 'reachmen'],
}

def get_cult(value):
    value = value.lower()
    v = [k for (k, v) in cult.items() if value in v]
    return v[0] if len(v) > 0 else value.title()

data.loc[:, "culture"] = [get_cult(x) for x in data.culture.fillna("")]

In [16]:
data_surv = data[(~data.age.isna()) & (data.age > 0)]  # Only using data with valid age values

map_bool = {1: False, 0: True}
data_surv_Y = [(map_bool[actual], age) for actual, age in zip(data_surv['actual'], data_surv['age'])]
data_surv_Y = [tuple(y) for y in data_surv_Y]
dt = np.dtype('?, int')
data_surv_Y = np.array(data_surv_Y, dt)

cols_dlt = ['actual', 'alive', 'name', 'plod', 'pred', 'isAlive', 'DateoFdeath', 'age', 'dateOfBirth', 'episodes', 'S.No']
data_surv_X = data_surv.drop(cols_dlt, 1)

for column in data_surv_X.columns:
    df = data_surv_X[~data_surv_X[column].isna()]
    if(len(df[column].value_counts()) <= 1):
        data_surv_X.drop([column], 1, inplace=True)            


data_surv_X.loc[:, "title"] = pd.factorize(data_surv_X.title)[0]
data_surv_X.loc[:, "house"] = pd.factorize(data_surv_X.house)[0]
data_surv_X.loc[:, "spouse"] = pd.factorize(data_surv_X.spouse)[0]
#data_surv_X.loc[:, "culture"] = pd.factorize(data_surv_X.spouse)[0]

for col in ['culture']:
    data_surv_X[col] = data_surv_X[col].astype('category') 

data_surv_X = OneHotEncoder().fit_transform(data_surv_X)  

data_surv_X.fillna(value = -1, inplace = True)

for column in data_surv_X:
    if(data_surv_X[column].astype(bool).sum(axis=0) <= 5):
        data_surv_X.drop([column], 1, inplace=True)


#data_surv_X['popularity'] = np.random.normal(0, 0.01, data_surv_X['popularity'].shape) + data_surv_X['popularity']


# Survival analysis
We have the data_X and the data_Y as required in scikit-survival, we can now start to do. Since we have multiple variables, we use [Cox's proportional hazards model](https://en.wikipedia.org/wiki/Proportional_hazards_model) using the [scikit-survival](https://scikit-survival.readthedocs.io/en/latest/) library

In [17]:
X_train, X_test, y_train, y_test = train_test_split(data_surv_X, data_surv_Y, test_size=0.2, random_state=62)

estimator_cox = CoxPHSurvivalAnalysis()
scores = cross_val_score(estimator_cox, X_train, y_train, cv=7)
estimator_cox.fit(X_train, y_train)

scores.mean()

0.6063895852785647

In [18]:
pd.Series(estimator_cox.coef_, index=X_train.columns)

title               -0.004286
male                 0.788561
culture=Dornish     -0.438544
culture=Free Folk    1.551378
culture=Ironborn    -0.809795
culture=Northmen     0.195294
culture=Rivermen    -1.555390
culture=Valemen      0.820255
culture=Valyrian     0.367391
culture=Westermen   -0.756967
culture=Westerosi   -1.630534
house               -0.000344
spouse              -0.004810
book1                0.263691
book2                0.634710
book3               -0.274606
book4               -0.581907
book5                0.233177
isAliveSpouse        0.022055
isMarried            0.338157
isNoble             -0.501305
numDeadRelations    -0.052483
boolDeadRelations    0.940090
isPopular           -0.005847
popularity           0.653401
screentime          -0.006939
dtype: float64

In [19]:
def fit_and_score_features(X, y):
    n_features = X.shape[1]
    scores = np.empty(n_features)
    m = CoxPHSurvivalAnalysis()
    for j in range(n_features):
        Xj = X[:, j:j+1]
        m.fit(Xj, y)
        scores[j] = m.score(Xj, y)
    return scores

scores = fit_and_score_features(X_train.values, y_train)

pipe = Pipeline([('select', SelectKBest(fit_and_score_features, k=3)),
                 ('model', CoxPHSurvivalAnalysis())])

param_grid = {'select__k': np.arange(1, X_train.shape[1] + 1)}
gcv = GridSearchCV(pipe, param_grid, return_train_score=True, cv=3, iid=True)
gcv.fit(X_train, y_train)

pd.DataFrame(gcv.cv_results_).sort_values(by='mean_test_score', ascending=False)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_select__k,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,mean_train_score,std_train_score
22,0.702113,0.105229,0.004972,0.000198,23,{'select__k': 23},0.553654,0.669069,0.560800,0.594388,0.052772,1,0.732940,0.698195,0.721160,0.717431,0.014427
21,0.691850,0.102365,0.005079,0.000256,22,{'select__k': 22},0.550879,0.662971,0.569128,0.594199,0.049092,2,0.728821,0.703084,0.710161,0.714022,0.010856
20,0.693189,0.103796,0.005012,0.000163,21,{'select__k': 21},0.546716,0.661308,0.572460,0.593358,0.049083,3,0.720308,0.699323,0.714286,0.711305,0.008822
19,0.704519,0.108674,0.005023,0.000203,20,{'select__k': 20},0.551341,0.661308,0.557468,0.589926,0.050426,4,0.722504,0.698822,0.714536,0.711954,0.009839
25,0.724150,0.135323,0.005128,0.000148,26,{'select__k': 26},0.553191,0.650222,0.556913,0.586677,0.044860,5,0.739805,0.695562,0.717785,0.717718,0.018062
23,0.698549,0.105671,0.005029,0.000220,24,{'select__k': 24},0.547179,0.641353,0.561355,0.583190,0.041444,6,0.734038,0.709227,0.719410,0.720892,0.010183
18,0.687591,0.101651,0.005019,0.000219,19,{'select__k': 19},0.550879,0.641353,0.556913,0.582954,0.041277,7,0.715914,0.701830,0.714536,0.710760,0.006339
24,0.697259,0.104729,0.005068,0.000107,25,{'select__k': 25},0.552729,0.628049,0.558023,0.579522,0.034307,8,0.739530,0.706093,0.720222,0.721948,0.013705
17,0.658352,0.097007,0.004977,0.000218,18,{'select__k': 18},0.548566,0.631929,0.555802,0.578678,0.037688,9,0.710284,0.695437,0.714411,0.706711,0.008148
16,0.512957,0.064490,0.001795,0.000025,17,{'select__k': 17},0.546253,0.630820,0.555247,0.577349,0.037906,10,0.708499,0.695312,0.713911,0.705907,0.007811


There does not seem to be any great insight in the table above. The best mean scores are for a large number of columns, so we just decide to keep them all

In [20]:
prediction = estimator_cox.predict(X_test)
result = concordance_index_censored(y_test["f0"], y_test["f1"], prediction)
result[0]

0.620429252782194

In [21]:
result_s = pd.concat([data_surv[['name', 'age']], data_surv_X], axis=1)
cols_to_keep = list(X_train.columns)
surv_by_name = {}

for name in result_s.name:
    row_test = result_s[result_s.name == name]
    row_test = row_test[cols_to_keep]
    surv_by_name.update({name : estimator_cox.predict_survival_function(row_test)[0]})
    
result_s['survival_fn'] = result_s.apply(lambda row: surv_by_name[row['name']], axis = 1)

### Taking a look at the results
We start by taking a look at predictions for the most influential characters, and whether they are expected to survive 5 years from the 'current date'

In [22]:
for name in result_s.nlargest(30, 'screentime').name:
    row = result_s[result_s.name == name]
    i = row.index[0]
    age = row.at[i, 'age']
    fn = row.at[i, 'survival_fn']
    print('{}(age = {}):'.format(name, age), fn(age+5) )

tyrion lannister(age = 32.0): 0.9338720195408754
jon snow(age = 22.0): 0.9130602979784419
daenerys targaryen(age = 21.0): 0.9615599636601011
cersei lannister(age = 39.0): 0.9570502315478597
sansa stark(age = 19.0): 0.948703198372795
arya stark(age = 16.0): 0.9493563966996424
jaime lannister(age = 39.0): 0.8565344698058327
theon greyjoy(age = 27.0): 0.7826085501183166
samwell tarly(age = 22.0): 0.9554774360798821
jorah mormont(age = 51.0): 0.3548877497250123
petyr baelish(age = 37.0): 0.39813502106476223
eddard stark(age = 36.0): 0.5764724792118956
davos seaworth(age = 45.0): 0.8768504389142265
bran stark(age = 15.0): 0.8324773853625015
catelyn stark(age = 35.0): 0.9764412271506899
tywin lannister(age = 58.0): 0.38446062932161656
margaery tyrell(age = 22.0): 0.7705630176002121
sandor clegane(age = 30.0): 0.44524331553005414
ramsay snow(age = 23.0): 0.7691548195926595
bronn(age = 41.0): 0.755548871928996
gilly(age = 23.0): 0.35120607822712324
ygritte(age = 19.0): 0.4799652327336576
shae(

**Conclusion for survival analysis**

We can see that the results are not great at all. Age seems to be the most important characteristic considered in calculating the probability of death, which makes sense. However, perhaps we did not have enough data to consider how the other characteristics would affect the survival, and we did not spend too much effort on tuning the model either. We also must keep in mind that the data is based on the fantasy world created in the mind of a single person which entertains party though surprising turns of events, which might inherently make it hard to find a pattern, or a general rule of survival.

# Binary classification
We also do a quick binary classification for the sake of completeness, since we ignored many characters in the survival analysis

In [23]:
data_y = data.actual.values

cols_dlt = ['actual', 'alive', 'name', 'plod', 'pred', 'isAlive', 'DateoFdeath', 'dateOfBirth', 'S.No']
data_X = data.drop(cols_dlt, 1)

for column in data_X.columns:
    df = data_X[~data_X[column].isna()]
    if(len(df[column].value_counts()) <= 1):
        data_X.drop([column], 1, inplace=True)            

cols_to_factor = [col for col in data_X.select_dtypes('object')] 
for col in cols_to_factor:
    data_X.loc[:, col] = pd.factorize(data_X[col])[0]

data_X.fillna(value = -1, inplace = True)

In [24]:
X_train, X_test, y_train, y_test = train_test_split(data_X, data_y, test_size=0.3, random_state=75)

In [25]:
estimator_lr = LogisticRegression(solver = 'lbfgs', max_iter = 10000)
scores_lr = cross_val_score(estimator_lr, X_train, y_train, cv=7)
estimator_lr.fit(X_train, y_train)
print(scores_lr.mean())

0.7885712821437626


This is alreay a seemingly much better result than we had with survival analysis

In [26]:
estimator_lr.score(X_test, y_test)

0.7654109589041096

In [28]:
result_lr = pd.concat([data[['name']], data_X], axis=1)
result_lr['prediction_lr'] = estimator_lr.predict(data_X)

for name in result_lr.nlargest(30, 'screentime').name:
    row = result_lr[result_lr.name == name]
    i = row.index[0]
    age = row.at[i, 'age']
    prediction = row.at[i, 'prediction_lr']
    print('{}(age = {}):'.format(name, age), prediction )

tyrion lannister(age = 32.0): 1
jon snow(age = 22.0): 1
daenerys targaryen(age = 21.0): 1
cersei lannister(age = 39.0): 1
sansa stark(age = 19.0): 1
arya stark(age = 16.0): 1
jaime lannister(age = 39.0): 1
theon greyjoy(age = 27.0): 1
samwell tarly(age = 22.0): 1
jorah mormont(age = 51.0): 1
petyr baelish(age = 37.0): 1
eddard stark(age = 36.0): 1
davos seaworth(age = 45.0): 1
bran stark(age = 15.0): 1
catelyn stark(age = 35.0): 1
varys(age = -1.0): 1
tywin lannister(age = 58.0): 0
margaery tyrell(age = 22.0): 0
robb stark(age = -1.0): 0
stannis baratheon(age = -1.0): 0
sandor clegane(age = 30.0): 1
joffrey baratheon(age = -1.0): 0
ramsay snow(age = 23.0): 1
melisandre(age = -1.0): 1
bronn(age = 41.0): 1
gilly(age = 23.0): 1
ygritte(age = 19.0): 1
shae(age = 20.0): 1
daario naharis(age = -1.0): 1
missandei(age = 17.0): 1


# Measures of fairness
We try to design a couple of simple measures of fairness to see the bias of gender in making the predictions

**1. Coeffecient for gender in the cox model**

The easiest to see is the coeffecient corresponding to the colum 'male' in the trained survival analysis model we had. In case of gender not being important to the prediction, the coefficient should be zero, since the coefficient is the log of the hazard (how more likely you are likely to die if the value of this variable is true)

In [29]:
coeff_gender = pd.Series(estimator_cox.coef_, index=X_train.columns)['male']

e = math.e
print('The hazard ratio of being male is {}'.format(e**(coeff_gender)))

The hazard ratio of being male is 2.2002279506551092


We can say that the farther this number if from 1, the more the bias is

**2. Expect 'similar' predictions for 'similar' features apart from gender**

Most measures of fairness will have a measure of similarity associated with them, to formalize who want things to be fair for, before we can formalize what we call fair. 

What we want in this problem is for the algorithm to predict death with similar probability for each of the groups 'male' and 'female', other things being equal. We can define 'other things being equal' in many ways. We could look at the titles of the characters, or the houses they belong to or what age group they are it, or  combination of these. We go with the following:

**Definition:** Given a fixed value of the attribute isPopular, the average of the predictions from the logistic regression model trained above should be similar for the groups with attributes gender=male and gender=female.

In [30]:
result_fair = result_lr[['isPopular', 'male', 'prediction_lr']]

In [31]:
result_fair.groupby(['isPopular', 'male']).mean()

prediction_lr
isPopular male               
0         0          0.960055
          1          0.919457
1         0          0.733333
          1          0.430000

We see that there is a significant difference in the avegrage expectation of death according the the regression model for popolar characters. However, without further analyis, it is hard to read much into this conclusion